## 1. Import Libraries

In [78]:
import os

## 2. Set Global Parameters

In [79]:
HIDDEN_LAYER_NEURONS = 10
OUTPUT_LAYER_NEURONS = 10

input_file_path = "../Training/model_weights.txt"  # Update this path to your weights file
output_directory = "./Hidden_Neuron_Weights/"  # Update this path to your desired output directory

dataWidth = 16
weightIntWidth = 3

outPATH = "Binary_Rep/"
inputFile = "Hidden_Neuron_Weights/"

weightFracWidth = dataWidth - weightIntWidth
print(weightFracWidth)

13


## 3. Utils Function for Separate All of Weights

In [80]:
def parse_weight_line(line):
    """
    Parse a line from the weight file and extract the layer, source neuron,
    destination neuron, and weight value. Adjusted to handle commas.
    """
    parts = line.replace(',', '').split()  # Remove commas before splitting
    layer = int(parts[2])
    source_neuron = int(parts[4])
    dest_neuron = int(parts[9])
    weight = float(parts[-1])
    return layer, source_neuron, dest_neuron, weight

In [81]:
def distribute_weights(input_file, output_dir):
    # Ensure the output directory exists
    if(not os.path.exists(output_dir)):
        os.makedirs(output_dir)
    
    # Adjust initialization to include neuron '0'
    neuron_weights = {'hidden': {i: [] for i in range(0, HIDDEN_LAYER_NEURONS + 1)},
                      'output': {i: [] for i in range(0, OUTPUT_LAYER_NEURONS + 1)}}
    
    # Read and parse the input file
    with open(input_file, 'r') as f:
        for line in f:
            layer, source_neuron, dest_neuron, weight = parse_weight_line(line)       
            neuron_type = 'hidden' if layer == 1 else 'output'
            neuron_weights[neuron_type][dest_neuron].append((source_neuron, weight))
    
    # Write weights to separate files
    for neuron_type, neurons in neuron_weights.items():
        for neuron_id, weights in neurons.items():
            file_name = f"{neuron_type}_neuron_{neuron_id}_weights.txt"
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, 'w') as f:
                for source_neuron, weight in weights:
                    f.write(f"{weight}\n")

## 4. Create Spare Weights File

In [82]:
distribute_weights(input_file_path, output_directory)

## 5. Convert Weights to the Binary Representation

### 5.1. To Binary Function

In [83]:
def DtoB(num, dataWidth, fracBits):
    if(num >= 0):
        num = num * (2 ** fracBits)
    else:
        num = (num + 2**(dataWidth + fracBits)) * (2 ** fracBits)
    num = int(num) & (2 ** dataWidth - 1)
    return format(num, '0{}b'.format(dataWidth))

In [84]:
files = [f for f in os.listdir(inputFile) if f.endswith('.txt')]
files

['hidden_neuron_0_weights.txt',
 'hidden_neuron_10_weights.txt',
 'hidden_neuron_1_weights.txt',
 'hidden_neuron_2_weights.txt',
 'hidden_neuron_3_weights.txt',
 'hidden_neuron_4_weights.txt',
 'hidden_neuron_5_weights.txt',
 'hidden_neuron_6_weights.txt',
 'hidden_neuron_7_weights.txt',
 'hidden_neuron_8_weights.txt',
 'hidden_neuron_9_weights.txt',
 'output_neuron_0_weights.txt',
 'output_neuron_10_weights.txt',
 'output_neuron_1_weights.txt',
 'output_neuron_2_weights.txt',
 'output_neuron_3_weights.txt',
 'output_neuron_4_weights.txt',
 'output_neuron_5_weights.txt',
 'output_neuron_6_weights.txt',
 'output_neuron_7_weights.txt',
 'output_neuron_8_weights.txt',
 'output_neuron_9_weights.txt']

### 5.2. Convert All Weights to Binary

In [86]:
for file in files:
    input_file_path = os.path.join(inputFile, file)
    output_file_path = os.path.join(outPATH + "txt/", f"bin_{file}")
    
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        for line in infile:
            num = float(line.strip())  # Read and convert each line to a float
            binary_rep = DtoB(num, dataWidth, weightFracWidth)  # Convert to binary
            outfile.write(binary_rep + '\n')  # Write the binary representation to the output file

print("All files have been processed and saved in 'Binary_Rep/' directory.")

All files have been processed and saved in 'Binary_Rep/' directory.


### 5.3. Save Weights With `.bat` Format 

In [87]:
for file in files:
    input_file_path = os.path.join(inputFile, file)
    output_file_path = os.path.join(outPATH + "bat/", f"bin_{os.path.splitext(file)[0]}.bat")  # Save as .bat
    
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        for line in infile:
            num = float(line.strip())  # Read and convert each line to a float
            binary_rep = DtoB(num, dataWidth, weightFracWidth)  # Convert to binary
            outfile.write(binary_rep + '\n')  # Write the binary representation to the output file

print("All files have been processed and saved in 'Binary_Rep/bat/' directory with .bat format.")

All files have been processed and saved in 'Binary_Rep/' directory with .bat format.
